# Word count with PySpark

#### Topics covered in this example
* Write a file to HDFS, read the file and perform word count on the data.

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* The EMR cluster attached to this notebook should have the `Spark` application installed.
* This notebook uses the `PySpark` kernel.
***

## Introduction
In this example we write a file to hdfs, use pyspark to count the occurrence of each word in the file stored in hdfs and store the results to s3.
***

## Setup
1. Create a S3 bucket to save your results or use an existing s3 bucket. For example: `s3://EXAMPLE-BUCKET/word-count/`
***

## Example

Create a test data frame with some sample records.
We will use the `createDataFrame()` method to create and `printSchema()` method to print out the schema.

In [ ]:
wordsDF = sqlContext.createDataFrame([("emr",), ("spark",), ("example",), ("spark",), ("pyspark",), ("python",),
             ("example",), ("emr",), ("example",), ("spark",), ("pyspark",), ("python",)], ['words'])
wordsDF.show()
wordsDF.printSchema()

Print out the number of unique words so that we can verify this number with the end result count.

In [ ]:
uniqueWordsCount = wordsDF.distinct().groupBy().count().head()[0]
print(uniqueWordsCount)

This step only shows an example on how to write to hdfs.
You can use an existing file stored in hdfs and read it as shown in the next steps.

In [ ]:
wordsDF.write.csv("hdfs:///user/hadoop/test-data.csv")

Read the csv file from hdfs and store in RDD.

In [ ]:
wordsData = sc.textFile("hdfs:///user/hadoop/test-data.csv")
wordsData.count()

Display the contents of the file.

In [ ]:
wordsData.collect()

Count the occurance of each word and print the count of the result. This should be equal to the number of unique words we found earlier.

In [ ]:
wordsCounts = wordsData.flatMap(lambda line: line.split(" ")) \
                       .map(lambda word: (word, 1)) \
                       .reduceByKey(lambda a, b: a+b)
wordsCounts.count()

Display the count for each word.

In [ ]:
wordsCounts.collect()

Save the results to your s3 bucket. The results are stored in the key `word-count` and split based on paritions.

In [ ]:
wordsCounts.saveAsTextFile("s3://EXAMPLE-BUCKET/word-count") # Change this to the S3 location that you created in Setup step 1.